# Word Embedding

**Word embedding** es el nombre de un conjunto de técnicas de modelado del lenguaje dadas dentro del Procesamiento del Lenguaje Natural (PLN), o *Natural Language Processing* en inglés, dónde las palabras o frases del vocabulario son vinculadas a vectores de números reales. Conceptualmente implica el encaje matemático de un espacio con una dimensión por palabra a un espacio vectorial continuo con menos dimensiones.

Algunos de los métodos para generar este mapeo son redes neuronales, reducción de dimensionalidad en la matriz de co-ocurrencia de palabras, modelos probabilísticos, y representación explícita en términos del contexto en el cual estas palabras figuran.

El *Word* y *phrase embeddings* (para palabras y frases respectivamente), utilizados de forma subyacente como forma de representación, demostraron aumentar el rendimiento de tareas en el procesamiento del lenguaje natural como en el análisis sintáctico y el análisis de sentimiento.

## Vector one-hot

Un vector *one-hot* consiste en un vector dónde sólo una de las posiciones tiene un valor, en este caso 1. Con él podemos representar un diccionario, donde cada palabra diferente se corresponde con un vector one-hot con una entrada diferente igual a 1.

### Ejemplo con *embedding* del tamaño del diccionario

En el siguiente ejemplo se codifican la palabras, dentro de un listado de frases, con un número entero equivalente a la entrada en el vector one-hot. El tamaño del vector debería ser igual al tamaño del diccionario. Sin embargo la función `one-hot()` de la librería `keras` no garantiza un número diferente para cada palabra, debido al método de hashing utilizado internamente, y por eso se usa un tamaño de vector mayor al número de palabras diferentes existentes para evitar colisiones.

Luego, las frases codificadas de esta manera, y con relleno, se pasan a una red neuronal que aprende a clasificarlas según nuestro propio etiquetado de frases positivas o negativas. La precisión obtenida es aproximadamente del 80%, pero que puede variar ya que la función `one-hot()` no devuelve siempre los mismos resultados (además de existir otros procesos aleatorios como la inicialización de pesos de la red). Véase: https://keras.io/preprocessing/text/ 

In [1]:
from numpy import array
from keras.preprocessing.text import one_hot
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Flatten
from keras.layers.embeddings import Embedding

# define documents
docs = ['Well done!',
        'Good work',
        'Great effort',
        'nice work',
        'Excellent!',
        'Weak',
        'Poor effort!',
        'not good',
        'poor work',
        'Could have done better.']

# define class labels. '1' means positive, '0' means negative.
labels = array([1,1,1,1,1,0,0,0,0,0])

# integer encode the documents
vocab_size = 50
encoded_docs = [one_hot(d, vocab_size) for d in docs]
print(encoded_docs)

Using TensorFlow backend.


[[39, 33], [18, 15], [36, 46], [25, 15], [32], [12], [43, 46], [23, 18], [43, 15], [21, 48, 33, 42]]


In [2]:
# pad documents to a max length of 4 words
max_length = 4
padded_docs = pad_sequences(encoded_docs, maxlen=max_length, padding='post')
print(padded_docs)

[[39 33  0  0]
 [18 15  0  0]
 [36 46  0  0]
 [25 15  0  0]
 [32  0  0  0]
 [12  0  0  0]
 [43 46  0  0]
 [23 18  0  0]
 [43 15  0  0]
 [21 48 33 42]]


In [3]:
# define the model
model = Sequential()
model.add(Embedding(vocab_size, 8, input_length=max_length))
model.add(Flatten())
model.add(Dense(1, activation='sigmoid'))

# compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['acc'])

# summarize the model
print(model.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 4, 8)              400       
_________________________________________________________________
flatten_1 (Flatten)          (None, 32)                0         
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 33        
Total params: 433
Trainable params: 433
Non-trainable params: 0
_________________________________________________________________
None


In [9]:
# fit the model
model.fit(padded_docs, labels, epochs=50, verbose=0)

# evaluate the model
loss, accuracy = model.evaluate(padded_docs, labels, verbose=0)
print('Accuracy: %f' % (accuracy*100))

Accuracy: 100.000000


## Word2vec

El *embedding* utilizado en el ejercicio anterior parece bastante mejorable. Por un lado el tamaño del mismo puede ser excesivo si tenemos un diccionario grande, ya que hay una entrada por cada palabra. ¿Se podría tener una representación más compacta? Por otro la política aleatoria para su inicialización (asignación de una palabra a una entrada del vector) es seguramente mejorable. ¿Existen políticas mejores donde la posición de las palabras dentro del vector tenga algún significado?

Con las técnicas **Word2vec** podemos conseguir un mejor *embedding* entrenando una sencilla red neuronal de dos capas con pares de palabras relacionadas. 

### Explicación práctica

Aquí se puede leer el tutorial entero: http://mccormickml.com/2016/04/19/word2vec-tutorial-the-skip-gram-model/

Los datos para entrenar nuestro modelo se pueden sacar fácilmente del texto disponible. Definimos un tamaño de ventana (un valor típico es 5) dentro del cual suponemos que hay relación entre las palabras y generamos todas las parejas posibles dentro de esa ventana. En la siguiente imagen se puede ver un ejemplo con tamaño de ventana 2.

<img src="images/training_data.png" width="60%"/>

Si tenemos un diccionario de 10.000 palabras la arquitectura de nuestra sencilla red neuronal sería como se ve en la imagen: vectores one-hot de tamaño 10.000 a la entrada, una única capa oculta de 300 neuronas (por ejemplo) y una capa de salida de 10.000 neuronas usando Softmax. Los valores de entrada serán el primer miembro de las parejas generadas anteriormente y el valor deseado el segundo miembro.

<img src="images/skip_gram_net_arch.png" width="80%"/>

Lo que conseguimos con esta arquitectura es que los pesos de la capa oculta, formada por las 300 neuronas, representen 300 características del diccionario. Los pesos aprendidos serán pues nuestro nuevo *embedding*.

<img src="images/word2vec_weight_matrix_lookup_table.png" width="50%"/>

A continuación se puede ver con un ejemplo reducido, como pasamos de una representación inicial de 5 valores (u vector one-hot de tamaño 5) a una representación compacta de sólo 3 valores. Esto se consigue gracias al entrenamiento de los pesos de una red oculta de 3 neuronas.

<img src="images/matrix_mult_w_one_hot.png" width="50%"/>

La salida de la red serán las probabilidades de que una palabra del diccionario esté relacionada con una palabra de entrada.

<img src="images/output_weights_function.png" width="80%"/>

En este enlace se pueden hacer consultas, sobre modelos previamente entrenados de finlandés, suomi e inglés, acerca de la relación entre palabras: http://bionlp-www.utu.fi/wv_demo/

### Técnicas *Continuous Bag-Of-Words* (CBOW) y *Continuous Skip-gram*

En Word2vec se pueden usar dos arquitecturas distintas para producir el *word embedding*:

1. **Continuous Bag-Of-Words** (CBOW): 
 - El modelo predice la palabra actual a partir de una ventana circundante de palabras del contexto.
 - El orden de las palabras del contexto no influye en la predicción.
 
2. **Continuous Skip-gram**:
 - El modelo usa la palabra actual para predecir la ventana circundante de palabras del contexto.
 - Las palabras del contexto más cercanas tienen más peso que las palabras más distantes.
 
*CBOW* es más rápido pero *Skip-gram* consigue mejores resultados para palabras poco frecuentes. 

### Parametrización

Los resultados de word2vec pueden ser sensibles a la parametrización. Algunos parámetros importantes son: 

* **Algoritmo de entrenamiento**: *Softmax jerárquico*, mejor para palabras poco frecuentes, o *Negative sampling*, mejor para palabras frecuentes y para vectores con menos dimensiones. A medida que el número de épocas aumenta el *Softmax jerárquico* deja de ser útil.

* **Sub-sampling de palabras frecuentes**: Las palabras muy frecuentes a menudo aportan poca información. Las palabras con una frecuencia de aparición por encima de un cierto umbral pueden ser sub-muestreadas para aumentar la velocidad de entrenamiento. También puede mejorar la precisión en datasets grandes. Valores útiles están en el rango 0,001 y 0,00001. 

* **Dimensionalidad del word embedding**: La calidad del *word embedding* suele incrementarse a mayor dimensión del vector, que se corresponde con el número de neuronas de la capa oculta. Pero a partir de cierto punto la ganancia es poco significativa. Valores típicos para la dimensionalidad están entre 100 y 1000.

* **Tamaño de la Ventana del contexto**: El tamaño de la ventana del contexto determina cuántas palabras antes y después de una palabra determinada deben ser incluídas dentro de su contexto. Valores recomendados son 10 para skip-gram y 5 para CBOW.

## GloVe

Training is performed on aggregated global word-word co-occurrence statistics from a corpus, and the resulting representations showcase interesting linear substructures of the word vector space. 

https://github.com/stanfordnlp/GloVe

The similarity metrics used for nearest neighbor evaluations produce a single scalar that quantifies the relatedness of two words. This simplicity can be problematic since two given words almost always exhibit more intricate relationships than can be captured by a single number. For example, man may be regarded as similar to woman in that both words describe human beings; on the other hand, the two words are often considered opposites since they highlight a primary axis along which humans differ from one another.

In order to capture in a quantitative way the nuance necessary to distinguish man from woman, it is necessary for a model to associate more than a single number to the word pair. A natural and simple candidate for an enlarged set of discriminative numbers is the vector difference between the two word vectors. GloVe is designed in order that such vector differences capture as much as possible the meaning specified by the juxtaposition of two words.

### Ejemplo con GloVe

In [6]:
from numpy import array
from numpy import asarray
from numpy import zeros
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Flatten
from keras.layers import Embedding

# define documents
docs = ['Well done!',
        'Good work',
        'Great effort',
        'nice work',
        'Excellent!',
        'Weak',
        'Poor effort!',
        'not good',
        'poor work',
        'Could have done better.']

# define class labels. '1' means positive, '0' means negative.
labels = array([1,1,1,1,1,0,0,0,0,0])

# prepare tokenizer
t = Tokenizer()
t.fit_on_texts(docs)
vocab_size = len(t.word_index) + 1

# integer encode the documents
encoded_docs = t.texts_to_sequences(docs)
print(encoded_docs)

# pad documents to a max length of 4 words
max_length = 4
padded_docs = pad_sequences(encoded_docs, maxlen=max_length, padding='post')
print(padded_docs)

[[6, 2], [3, 1], [7, 4], [8, 1], [9], [10], [5, 4], [11, 3], [5, 1], [12, 13, 2, 14]]
[[ 6  2  0  0]
 [ 3  1  0  0]
 [ 7  4  0  0]
 [ 8  1  0  0]
 [ 9  0  0  0]
 [10  0  0  0]
 [ 5  4  0  0]
 [11  3  0  0]
 [ 5  1  0  0]
 [12 13  2 14]]


In [10]:
# load the whole embedding into memory
embeddings_index = dict()
f = open('./GloVe/glove.6B.100d.txt', encoding="utf8")
for line in f:
    values = line.split()
    word = values[0]
    coefs = asarray(values[1:], dtype='float32')
    embeddings_index[word] = coefs
f.close()
print('Loaded %s word vectors.' % len(embeddings_index))

# create a weight matrix for words in training docs
embedding_matrix = zeros((vocab_size, 100))
for word, i in t.word_index.items():
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        embedding_matrix[i] = embedding_vector

# define model
model = Sequential()
e = Embedding(vocab_size, 100, weights=[embedding_matrix], input_length=4, trainable=False)
model.add(e)
model.add(Flatten())
model.add(Dense(1, activation='sigmoid'))

# compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['acc'])

# summarize the model
print(model.summary())

# fit the model
model.fit(padded_docs, labels, epochs=50, verbose=0)

# evaluate the model
loss, accuracy = model.evaluate(padded_docs, labels, verbose=0)
print('Accuracy: %f' % (accuracy*100))

FileNotFoundError: [Errno 2] No such file or directory: './GloVe/glove.6B.100d.txt'